In [26]:
import requests
import pandas as pd

In [27]:
response = requests.get("https://www.stocklore.ai/api/nse_data.csv?q=TCS")
response.status_code

200

In [44]:
a = response.content.decode("utf-8").split("\n")
a = [i.split(",") for i in a]

In [45]:
a

[['Date', 'Open', 'Close', 'High', 'Low', 'Volume'],
 ['2008-01-01', '1077.0', '1054.55', '1077.0', '1052.0', '232097'],
 ['2008-01-02', '1060.0', '1050.0', '1061.0', '1031.5', '716231'],
 ['2008-01-03', '1030.1', '1011.05', '1045.0', '1005.35', '1292070'],
 ['2008-01-04', '1011.05', '1005.2', '1022.9', '1002.3', '579717'],
 ['2008-01-07', '1000.0', '975.6', '1008.8', '972.5', '911836'],
 ['2008-01-08', '980.0', '987.85', '992.0', '969.3', '801933'],
 ['2008-01-09', '994.8', '996.3', '1006.0', '989.9', '612532'],
 ['2008-01-10', '998.0', '981.65', '1009.5', '962.0', '835594'],
 ['2008-01-11', '915.35', '989.75', '1004.0', '915.35', '537297'],
 ['2008-01-14', '999.8', '961.15', '999.8', '955.6', '1039716'],
 ['2008-01-15', '965.0', '940.0', '968.0', '934.0', '733897'],
 ['2008-01-16', '940.0', '944.95', '966.7', '925.0', '782989'],
 ['2008-01-17', '965.8', '922.7', '966.0', '919.0', '1575524'],
 ['2008-01-18', '929.5', '903.2', '940.0', '897.1', '1083454'],
 ['2008-01-21', '904.0', '831

In [49]:
df = pd.DataFrame(a[1:], columns=a[0])
df

,Date,Open,Close,High,Low,Volume
0,2008-01-01,1077.0,1054.55,1077.0,1052.0,232097
1,2008-01-02,1060.0,1050.0,1061.0,1031.5,716231
2,2008-01-03,1030.1,1011.05,1045.0,1005.35,1292070
3,2008-01-04,1011.05,1005.2,1022.9,1002.3,579717
4,2008-01-07,1000.0,975.6,1008.8,972.5,911836
5,2008-01-08,980.0,987.85,992.0,969.3,801933
6,2008-01-09,994.8,996.3,1006.0,989.9,612532
7,2008-01-10,998.0,981.65,1009.5,962.0,835594
8,2008-01-11,915.35,989.75,1004.0,915.35,537297
9,2008-01-14,999.8,961.15,999.8,955.6,1039716
